In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo(id="QjzMFdrVeF4", width="100%")


In [ ]:

import pandas as pd
url="https://gist.githubusercontent.com/mamonu/01c64bc67e83cdd88b4fab5edb017a93/raw/fb2798f8d8d1a00ff58cf1fec3fb1586ec29d857/spies.csv"

In [ ]:
spies = pd.read_csv(url,header=None).rename(columns={1: "src", 2: "dst"}).drop([0],axis=1)
spies.head(100)

In [ ]:
import networkx as nx

In [ ]:
G = nx.from_pandas_edgelist(spies,"src","dst")

In [ ]:

pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos, node_color= ["black"],node_size=100)

In [ ]:
from nxviz import MatrixPlot

m = MatrixPlot(G)
m.draw()

In [ ]:
from nxviz import CircosPlot

c = CircosPlot(G,)
c.draw()

In [ ]:
nx.number_connected_components(G)

In [ ]:
G_comp  = [G.subgraph(c).copy() for c in nx.connected_components(G)]

In [ ]:
nx.draw(G_comp[0],pos)

In [ ]:
nx.draw(G_comp[1],pos)

In [ ]:
nx.draw(G_comp[2],pos)

In [ ]:
from networkx.algorithms import *

In [ ]:
hc = harmonic_centrality(G)


#The closeness centrality (or closeness) of a node is the average length of the shortest path between the node 
#and all other nodes in the graph. 
#Thus the more central a node is, the closer it is to all other nodes.

cc=closeness_centrality(G)


#Degree centrality is the number of links incident upon a node (i.e., the number of ties that a node has)
dc=degree_centrality(G)

#Betweenness centrality quantifies the number of times a node acts as a bridge along the shortest path 
#between two other nodes. 
bc=betweenness_centrality(G)


In [ ]:
def get_top_keys(dictionary, top):
    items = dictionary.items()
    items = sorted(items,reverse=True, key=lambda x: x[1])
    return list(map(lambda x: x[0], items[:top]))

In [ ]:
nodes = list(G.nodes)
counts = nx.number_connected_components(G)
output = []
for i, node in enumerate(nodes):
    subG = G.subgraph(nodes[:i]+nodes[i+1:])
    subGCount = nx.number_connected_components(subG)
    if subGCount>counts:
        output.append((node,1))
    else:
        output.append((node,0))

articulationdf= pd.DataFrame(output,columns=["nodeid","articulation"])
artdf=articulationdf.query("articulation==1")

In [ ]:
articulation_color_map=[]
for node in G:
    #print(node)
    if node in artdf.nodeid.values:
        articulation_color_map.append('red')
    else: 
        articulation_color_map.append('yellow')  

In [ ]:
for index, row in spies.iterrows():
    G[row['src']][row['dst']]['color'] = 'b' 
    
edges = G.edges()
edgecolors = [G[u][v]['color'] for u,v in edges]

In [ ]:





pos = nx.kamada_kawai_layout(G,scale=2,dim=2)
nx.draw(G, pos, node_color= articulation_color_map,edge_color=edgecolors,node_size=100)


In [ ]:


closenodes = get_top_keys(cc,10)

closeness_color_map=[]
for node in G:
    #print(node)
    if node in closenodes:
        closeness_color_map.append('green')
    else: 
        closeness_color_map.append('yellow')  

In [ ]:
nx.draw(G, pos, node_color= closeness_color_map,edge_color=edgecolors,node_size=100)

In [ ]:

btwnodes = get_top_keys(cc,10)

btwn_color_map=[]
for node in G:
    #print(node)
    if node in btwnodes:
        btwn_color_map.append('cyan')
    else: 
        btwn_color_map.append('yellow')  
        
nx.draw(G, pos, node_color= btwn_color_map,edge_color=edgecolors,node_size=100)

In [ ]:
def avg_neigh_degree(g):
    avgndeg = {}
    for n in g.nodes():
        if g.degree(n):
            avgndeg[n] = float(sum(g.degree(i) for i in g[n]))/g.degree(n)
    return avgndeg

In [ ]:

avgnnodes = get_top_keys(avg_neigh_degree(G),10) 
avgn_color_map=[]
for node in G:
    #print(node)
    if node in avgnnodes:
        avgn_color_map.append('black')
    else: 
        avgn_color_map.append('yellow')  
        
nx.draw(G, pos, node_color= avgn_color_map,edge_color=edgecolors,node_size=100)

In [ ]:
pd.DataFrame.from_dict(bc,orient="index",columns=["btwn_centrality"]).\
reset_index().rename(columns={"index": "node", "btwn_centrality": "btwn_centrality"})

In [ ]:
## METRICS and Processing for edges

In [ ]:
bridgesdf = pd.DataFrame(nx.bridges(G),columns=["src","dst"])

In [ ]:
bridgesdf

In [ ]:
eb = nx.edge_betweenness_centrality(G)
eb

In [ ]:

srclist = []
dstlist = []
eblist = []

for srcdst, v in eb.items():
        # unpack (src,dst) tuple key
        src, dst = srcdst

        srclist.append(src)
        dstlist.append(dst)
        eblist.append(v)

ebdf=pd.DataFrame(zip(srclist, dstlist, eblist), columns=["src", "dst", "eb"])
ebdf

In [ ]:
ebvalues=ebdf.eb.values
maxeb=max(ebvalues)
mineb=min(ebvalues)
zvalues = [float(x-mineb)/(maxeb-mineb)*10 for x in ebvalues]
print(max(zvalues),min(zvalues))


In [ ]:
nx.draw(G, pos, node_color= avgn_color_map,edge_color='grey',node_size=50,width=zvalues)